In [13]:
# Libraries for selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
# Waited conditionals for Selenium
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Data Manipulations libraries
import pandas as pd

# Regular expressions library
import re

# Datetime library
import time

In [14]:
# Open the web browser
driver = webdriver.Edge()
# Maximize the window
driver.maximize_window()

In [15]:
# Acceder al SIGMAA
url = 'https://uclb.ucaribe.edu.mx/sigmaav2/'
driver.get(url)

In [16]:
# Biblioteca para leer archivos
import os
# Biblioteca para leer .env
from dotenv import load_dotenv
# Cargamos variables de ambiente
load_dotenv()

# Obtener las credenciales del estudiante
username = os.getenv('ID')
password = os.getenv('PASS')

In [17]:
# Buscar el campo de usuario y escribir el username
userinput = driver.find_element(By.XPATH, '/html/body/div[2]/form/div/span[2]/input')
userinput.send_keys(username)
# Buscar el campo de contraseña y escribir el password
passinput = driver.find_element(By.XPATH, '/html/body/div[2]/form/div/input')
passinput.send_keys(password)
# Buscar el botón de submit y dar clic para iniciar sesión
submitinput = driver.find_element(By.XPATH, '/html/body/div[2]/form/button')
submitinput.click()

In [18]:
sistemaPagos = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div/div/div/ul[2]/li[7]/a')
sistemaPagos.click()

In [19]:
# Extraer la tabla usando webscrapping
dfs = pd.read_html(driver.page_source)

In [20]:
pagos = dfs[0].drop(['Opciones', 'Opciones.1'], axis = 1)

In [21]:
# Importar biblioteca para conexion a base de datos
from sqlalchemy import create_engine
# Establecer conexion a base de datos
engine = create_engine("mysql+mysqldb://root:@localhost/sure")

In [22]:
nombre_tabla = "pagos_" + username
pagos.to_sql(nombre_tabla, con = engine, if_exists='replace')

14

In [23]:
# Cerrar sesión
logout = driver.find_element(By.XPATH, '/html/body/div[2]/div/form/a')
logout.click()
# Cerrar el navegador
driver.quit()